In [ ]:
# Importamos las librerías necesarias
import pandas as pd # Para el manejo de datos
import os # Para manejar rutas de archivos
import glob # Para buscar archivos con patrones específicos

In [ ]:
# Definimos la ruta de carpeta de datos 
ruta_carpeta = '../../data/raw_data/students_by_municipality/'

# Obtenemos la lista de archivos CSV en la carpeta
lista_archivos = glob.glob(os.path.join(ruta_carpeta, '*.csv'))

# Lista de DF final 
df_a_unir = [] 
# Iteramos sobre cada archivo y lo procesamos
for archivo in lista_archivos:
    try: 
        # leemos el archivo csv
        df_temp = pd.read_csv(archivo)
        # Filtramos por localidad 0 (municipio)
        df_temp = df_temp[df_temp['LOC'] == 0].copy()
        # eliminamos el primer registro 
        df_temp_limpio = df_temp.iloc[1:]
        # añadimos a la lista 
        df_a_unir.append(df_temp_limpio)
    except Exception as e:
        print(f'Error al procesar el archivo {archivo}: {e}')

# Unimos todos los dataframes en uno solo
df_estudiantes_municipio = pd.concat(df_a_unir, ignore_index=True)


In [ ]:
# Mostramos las primeras filas del DataFrame resultante
df_estudiantes_municipio.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,P18A24A
0,11,Guanajuato,1,Abasolo,0,Total del Municipio,2701
1,11,Guanajuato,2,Acámbaro,0,Total del Municipio,3084
2,11,Guanajuato,3,San Miguel de Allende,0,Total del Municipio,4513
3,11,Guanajuato,4,Apaseo el Alto,0,Total del Municipio,1931
4,11,Guanajuato,5,Apaseo el Grande,0,Total del Municipio,3126


In [ ]:
# Mostramos un resumen de los datos cargados
print(f"{df_estudiantes_municipio['NOM_ENT'].unique()}, "
      f"{len(df_estudiantes_municipio['NOM_ENT'].unique())} estados representados, "
      f"con un total de {df_estudiantes_municipio.shape[0]} registros")

['Guanajuato' 'Guerrero' 'Hidalgo' 'Jalisco' 'México'
 'Michoacán de Ocampo' 'Morelos' 'Nayarit' 'Nuevo León' 'Oaxaca' 'Puebla'
 'Querétaro' 'Quintana Roo' 'San Luis Potosí' 'Sinaloa' 'Sonora' 'Tabasco'
 'Tamaulipas' 'Tlaxcala' 'Veracruz de Ignacio de la Llave' 'Yucatán'
 'Zacatecas' 'Aguascalientes' 'Baja California Sur' 'Baja California'
 'Campeche' 'Chiapas' 'Chihuahua' 'Ciudad de México'
 'Coahuila de Zaragoza' 'Colima' 'Durango'], 32 estados representados, con un total de 2469 registros


In [ ]:
# Cargamos el conjunto de datos del censo de los estados unidos mexicanos
ruta_estados_mexicanos = '..\\data\\censo_estados_unidos_mexicanos\\iter_00_cpv2020\\conjunto_de_datos\\conjunto_de_datos_iter_00CSV20.csv' 
df_estados_u_mexicanos = pd.read_csv(ruta_estados_mexicanos)
df_estados_u_mexicanos = df_estados_u_mexicanos[['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'POBTOT', 'P_18A24']].copy()
df_estados_u_mexicanos.head()

C:\Users\bruno\AppData\Local\Temp\ipykernel_24336\683626159.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estados_u_mexicanos = pd.read_csv(ruta_estados_mexicanos)


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,POBTOT,P_18A24
0,0,Total nacional,0,Total nacional,0,Total nacional,126014024,14736111
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,250354,27841
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,147125,16336
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,1425607,180847
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,3697,438


In [7]:
# Nos quedamos solo con las LOC == 0 
df_estados_u_mexicanos = df_estados_u_mexicanos[(df_estados_u_mexicanos['LOC'] == 0) & (df_estados_u_mexicanos['NOM_LOC'] == 'Total del Municipio')].copy()
df_estados_u_mexicanos.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,POBTOT,P_18A24
6,1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,948990,120452
574,1,Aguascalientes,2,Asientos,0,Total del Municipio,51536,6581
749,1,Aguascalientes,3,Calvillo,0,Total del Municipio,58250,6935
918,1,Aguascalientes,4,Cosío,0,Total del Municipio,17000,2228
985,1,Aguascalientes,5,Jesús María,0,Total del Municipio,129929,16640


In [ ]:
# Realizamos el merge entre los dos dataframes
df_merged = pd.merge(df_estudiantes_municipio, df_estados_u_mexicanos, on=['NOM_ENT','NOM_MUN', 'NOM_LOC', 'MUN'], how='left')
columnas_interes = ['NOM_ENT','NOM_MUN','NOM_LOC','POBTOT','P18A24A']
df_merged_final = df_merged[columnas_interes].copy()

# Reemplazamos el valor nulo población específica de Oaxaca en la columna POBTOT
if df_merged_final['POBTOT'].isnull().any():
    df_merged_final['POBTOT'].fillna(11465, inplace=True)

df_merged_final.isnull().sum()


In [12]:
df_estados_u_mexicanos.to_csv('..\\data\\estudiantes_municipio_todos_los_estados.csv', index=False)